In [1]:
# import goodies
import pandas as pd
import requests
import csv
import time
import os

data = pd.read_csv(r"C:\Users\toirt\LeagueOfLegendsPrediction\data\match_ids.txt", delimiter='\t', header = None)
df = pd.DataFrame(data)
df.columns = ['match_id']
df.shape # 71675 matches
df.head()

,match_id
0,NA1_4924378131
1,NA1_4924264206
2,NA1_4923951725
3,NA1_4923834112
4,NA1_4923514356


In [2]:
# Convert to list
match_ids = df['match_id'].tolist()

In [3]:
api_key = 'INSERT HERE'
api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches'
file_path = r"C:\Users\toirt\LeagueOfLegendsPrediction\data\match_data.csv"

def get_match_json(api_key, api_url, match_id):
    '''
    Get match data from match_id
    '''
    match_url = '%s/%s?api_key=%s' %(api_url, match_id, api_key)
    while True:
        resp = requests.get(match_url)
        
        if resp.status_code == 429:
            time.sleep(10)
            continue
    
        match_json = resp.json()
        return match_json

In [45]:
def process_match_json(game):
    '''
    Takes in match json and converts to dataframe with relevant features
    '''
    # Create blank DataFrame
    df = pd.DataFrame()
    # Feature definition
    info = game['info']
    players = info['participants']
    teams = info['teams']

    gameDuration = info['gameDuration']
    gameVersion = info['gameVersion']

    for player in players:
        # Player stats
        summonerName = player['summonerName']
        teamId = player['teamId']
        lane = player['lane']
        role = player['role']
        summonerLevel  = player['summonerLevel']
        teamPosition = player['teamPosition']

        # Champion stats
        champExperience = player['champExperience']
        champLevel = player['champLevel']
        championName = player['championName']

        # KDA
        kills = player['kills']
        deaths = player['deaths']
        assists = player['assists']
        doubleKills = player['doubleKills']
        tripleKills = player['tripleKills']
        quadraKills = player['quadraKills']
        pentaKills = player['pentaKills']
        largestKillingSpree = player['largestKillingSpree']
        largestMultiKill = player['largestMultiKill']
        killingSprees = player['killingSprees']
        firstBloodAssist = player['firstBloodAssist']
        firstBloodKill = player['firstBloodKill']
        unrealKills = player['unrealKills']

        # Damage Dealt
        totalDamageDealt = player['totalDamageDealt']
        totalDamageDealtToChampions = player['totalDamageDealtToChampions']
        totalDamageShieldedOnTeammates = player['totalDamageShieldedOnTeammates']
        totalDamageTaken = player['totalDamageTaken']
        totalHeal = player['totalHeal']
        trueDamageDealt = player['trueDamageDealt']
        trueDamageDealtToChampions = player['trueDamageDealtToChampions']
        trueDamageTaken = player['trueDamageTaken']
        magicDamageDealt = player['magicDamageDealt']
        magicDamageDealtToChampions = player['magicDamageDealtToChampions']
        magicDamageTaken = player['magicDamageTaken']
        physicalDamageDealt = player['physicalDamageDealt']
        physicalDamageDealtToChampions = player['physicalDamageDealtToChampions']
        physicalDamageTaken = player['physicalDamageTaken']

        # Objectives
        baronKills = player['baronKills']
        damageDealtToBuildings = player['damageDealtToBuildings']
        damageDealtToObjectives = player['damageDealtToObjectives']
        damageDealtToTurrets = player['damageDealtToTurrets']
        objectivesStolen = player['objectivesStolen']
        objectivesStolenAssists = player['objectivesStolenAssists']
        dragonKills = player['dragonKills']
        inhibitorKills = player['inhibitorKills']
        inhibitorTakedowns = player['inhibitorTakedowns']
        inhibitorsLost = player['inhibitorsLost']
        turretKills = player['turretKills']
        turretTakedowns = player['turretTakedowns']
        turretsLost = player['turretsLost']
        firstTowerAssist = player['firstTowerAssist']
        firstTowerKill = player['firstTowerKill']

        # Gold income
        goldEarned = player['goldEarned']
        goldSpent = player['goldSpent']
        # CS
        neutralMinionsKilled = player['neutralMinionsKilled']
        totalMinionsKilled = player['totalMinionsKilled']

        # Items
        item0 = player['item0']
        item1 = player['item1']
        item2 = player['item2']
        item3 = player['item3']
        item4 = player['item4']
        item5 = player['item5']
        item6 = player['item6']
        itemsPurchased = player['itemsPurchased']

        # Vision
        detectorWardsPlaced = player['detectorWardsPlaced']
        visionScore = player['visionScore']
        visionWardsBoughtInGame = player['visionWardsBoughtInGame']
        wardsKilled = player['wardsKilled']
        wardsPlaced = player['wardsPlaced']
        sightWardsBoughtInGame = player['sightWardsBoughtInGame']

        # Time
        longestTimeSpentLiving = player['longestTimeSpentLiving']
        timeCCingOthers = player['timeCCingOthers']
        timePlayed = player['timePlayed']
        totalTimeCCDealt = player['totalTimeCCDealt']
        totalTimeSpentDead = player['totalTimeSpentDead']

        # Game Outcomes
        gameEndedInEarlySurrender = player['gameEndedInEarlySurrender']
        gameEndedInSurrender = player['gameEndedInSurrender']
        teamEarlySurrendered = player['teamEarlySurrendered']
        win = player['win']
        
        teamColor = None # init team color
        for team in teams:
            if team['teamId'] == player['teamId']:
                objs = team['objectives']
                baron = objs['baron']
                dragon = objs['dragon']
                grubs = objs['horde']
                riftHerald =['riftHerald']
                tower = objs['tower']
                inhibitor = objs['inhibitor']
                if team['teamId'] == 100:
                    teamColor = 'red'
                else:
                    teamColor = 'blue'

        # DataFrame creation
        match_df = pd.DataFrame({
            'teamColor': [teamColor],
            'gameDuration': [gameDuration],
            'gameVersion': [gameVersion],
            'objectives': [objs],
            'baron': [baron],
            'dragon': [dragon],
            'grubs': [grubs],
            'riftHerald': [riftHerald],
            'tower': [tower],
            'inhibitor': [inhibitor],
            'summonerName': [summonerName],
            'lane': [lane],
            'role': [role],
            'summonerLevel': [summonerLevel],
            'teamPosition': [teamPosition],
            'champExperience': [champExperience],
            'champLevel': [champLevel],
            'championName': [championName],
            'kills': [kills],
            'deaths': [deaths],
            'assists': [assists],
            'doubleKills': [doubleKills],
            'tripleKills': [tripleKills],
            'quadraKills': [quadraKills],
            'pentaKills': [pentaKills],
            'largestKillingSpree': [largestKillingSpree],
            'largestMultiKill': [largestMultiKill],
            'killingSprees': [killingSprees],
            'firstBloodAssist': [firstBloodAssist],
            'firstBloodKill': [firstBloodKill],
            'unrealKills': [unrealKills],
            'totalDamageDealt': [totalDamageDealt],
            'totalDamageDealtToChampions': [totalDamageDealtToChampions],
            'totalDamageShieldedOnTeammates': [totalDamageShieldedOnTeammates],
            'totalDamageTaken': [totalDamageTaken],
            'totalHeal': [totalHeal],
            'trueDamageDealt': [trueDamageDealt],
            'trueDamageDealtToChampions': [trueDamageDealtToChampions],
            'trueDamageTaken': [trueDamageTaken],
            'magicDamageDealt': [magicDamageDealt],
            'magicDamageDealtToChampions': [magicDamageDealtToChampions],
            'magicDamageTaken': [magicDamageTaken],
            'physicalDamageDealt': [physicalDamageDealt],
            'physicalDamageDealtToChampions': [physicalDamageDealtToChampions],
            'physicalDamageTaken': [physicalDamageTaken],
            'baronKills': [baronKills],
            'damageDealtToBuildings': [damageDealtToBuildings],
            'damageDealtToObjectives': [damageDealtToObjectives],
            'damageDealtToTurrets': [damageDealtToTurrets],
            'objectivesStolen': [objectivesStolen],
            'objectivesStolenAssists': [objectivesStolenAssists],
            'dragonKills': [dragonKills],
            'inhibitorKills': [inhibitorKills],
            'inhibitorTakedowns': [inhibitorTakedowns],
            'inhibitorsLost': [inhibitorsLost],
            'turretKills': [turretKills],
            'turretTakedowns': [turretTakedowns],
            'turretsLost': [turretsLost],
            'firstTowerAssist': [firstTowerAssist],
            'firstTowerKill': [firstTowerKill],
            'goldEarned': [goldEarned],
            'goldSpent': [goldSpent],
            'neutralMinionsKilled': [neutralMinionsKilled],
            'totalMinionsKilled': [totalMinionsKilled],
            'item0': [item0],
            'item1': [item1],
            'item2': [item2],
            'item3': [item3],
            'item4': [item4],
            'item5': [item5],
            'item6': [item6],
            'itemsPurchased': [itemsPurchased],
            'detectorWardsPlaced': [detectorWardsPlaced],
            'visionScore': [visionScore],
            'visionWardsBoughtInGame': [visionWardsBoughtInGame],
            'wardsKilled': [wardsKilled],
            'wardsPlaced': [wardsPlaced],
            'sightWardsBoughtInGame': [sightWardsBoughtInGame],
            'longestTimeSpentLiving': [longestTimeSpentLiving],
            'timeCCingOthers': [timeCCingOthers],
            'timePlayed': [timePlayed],
            'totalTimeCCDealt': [totalTimeCCDealt],
            'totalTimeSpentDead': [totalTimeSpentDead],
            'gameEndedInEarlySurrender': [gameEndedInEarlySurrender],
            'gameEndedInSurrender': [gameEndedInSurrender],
            'teamEarlySurrendered': [teamEarlySurrendered],
            'win': [win]
        })

        # add to current df    
        df = pd.concat([df, match_df])

        
    return df

In [46]:
# Write to csv function
def write_df_to_csv(df):
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        df.to_csv(file_path, mode='w', index=False)  # Write with header
    else:
        df.to_csv(file_path, mode='a', header=False, index=False)

In [49]:
i = 0
for match_id in match_ids[600:1000]:
    i += 1
    try:
        match_json = get_match_json(api_key, api_url, match_id)
        df = process_match_json(match_json)
        write_df_to_csv(df)
        print(f"Iteration: {i}/{len(match_ids)}", end='\r')
    except KeyError:
        print('error')



UnboundLocalError: cannot access local variable 'objs' where it is not associated with a value